In [127]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional,RepeatVector,TimeDistributed
import numpy as np

In [134]:
# This is uinverient LSTM
def getData(lim=10,steps=3,function=None):
    if function is None:
        seq = np.arange(lim)*10
    else:
        seq = function(np.arange(lim))
    x = []
    y = []
    for i in range(len(seq)):
        if i + steps + 1 >= lim:
            break
        temp = []
        for j in range(steps):
            temp.append(seq[i+j])
        x.append(temp)
        y.append(seq[i+steps])
    return np.asarray(x),np.asarray(y)

def getTestData(func,nSamples=1,nSteps=4,uLimit=1000):
    data = []
    res = []
    for _ in range(nSamples):
        stPoint = np.random.randint(0,uLimit)
        temp = [func(stPoint+i) for i in range(nSteps)]
        data.append(temp)
        res.append(func(stPoint+nSteps))
    return np.expand_dims(np.asarray(data),axis=-1),np.asarray(res)
func = lambda x: x*x + 2*x + 8
n_steps=4
features=1
lim = 100
x,y = getData(lim, n_steps,func)

In [131]:
model = Sequential()
# ,return_sequences=True
model.add(LSTM(50, activation="relu", return_sequences=True, input_shape=(n_steps, features)))
model.add(LSTM(50, activation="relu", return_sequences=True))
model.add(LSTM(50, activation="relu"))
model.add(Dense(1))
model.compile(loss="mse",optimizer="adam")

In [135]:
samples,timesteps = x.shape
featues = 1

In [136]:
print(x.shape)

(95, 4)


In [137]:
x = x.reshape((samples,timesteps,features))

In [139]:
model.fit(x,y,epochs=200)

Epoch 1/200
3/3 [==============================] - 0s 26ms/step - loss: 20752142.0000
Epoch 2/200
3/3 [==============================] - 0s 27ms/step - loss: 20506160.0000
Epoch 3/200
3/3 [==============================] - 0s 26ms/step - loss: 20584636.0000
Epoch 4/200
3/3 [==============================] - 0s 30ms/step - loss: 20032728.0000
Epoch 5/200
3/3 [==============================] - 0s 28ms/step - loss: 19811090.0000
Epoch 6/200
3/3 [==============================] - 0s 27ms/step - loss: 19327566.0000
Epoch 7/200
3/3 [==============================] - 0s 27ms/step - loss: 19023842.0000
Epoch 8/200
3/3 [==============================] - 0s 32ms/step - loss: 18511406.0000
Epoch 9/200
3/3 [==============================] - 0s 28ms/step - loss: 18132014.0000
Epoch 10/200
3/3 [==============================] - 0s 28ms/step - loss: 17517462.0000
Epoch 11/200
3/3 [==============================] - 0s 31ms/step - loss: 16536226.0000
Epoch 12/200
3/3 [==============================] - 

3/3 [==============================] - 0s 26ms/step - loss: 1.0128


In [140]:
nSamples=1000
uLimit=200
test = getTestData(func,nSamples=nSamples,uLimit=uLimit)
tx,ty = test
res=model.predict(tx)
print(np.average(abs(res.flatten()-ty)))

263.0260894088745


In [101]:
# This is multi-steps LSTM data prepration

def getMSData(lim=10,stepsIn=3, stepsOut=2, function=None):
    if function is None:
        seq = np.arange(lim)*10
    else:
        seq = function(np.arange(lim))
    x = []
    y = []
    for i in range(len(seq)):
        if i + stepsIn + 1 >= lim:
            break
        temp = []
        for j in range(stepsIn):
            temp.append(seq[i+j])
        x.append(temp)
        y.append(seq[i+stepsIn:i+stepsIn+stepsOut])
    
    from sklearn.utils import shuffle
    x,y=np.asarray(x),np.asarray(y)
    x,y = shuffle(x,y)
    return x.reshape((-1,3,1)),y

def getTestMSData(func=None,nSamples=1,nSteps=3,stepsOut=2,uLimit=100):
    data = []
    res = []
    for _ in range(nSamples):
        stPoint = np.random.randint(0,uLimit)
        if func is not None:
            temp = [func(stPoint+i) for i in range(nSteps)]
            data.append(temp)
            res.append([func(stPoint+nSteps+i) for i in range(stepsOut)])
        else:
            temp = (np.arange(nSteps)+stPoint)*10
            data.append(temp)
            res.append((np.arange(stepsOut)+stPoint+nSteps)*10)
    return np.expand_dims(np.asarray(data),axis=-1),np.asarray(res)

def Generator(bachSize=100, uLimit=2000,func=None):
    while True:
        yield getTestMSData(nSamples=batchSize,uLimit=uLimit,func=func)

In [112]:
# x,y = getMSData(lim=1000)
# xval,yval = getTestMSData(nSamples=100,uLimit=1200)
# features = 1
# samples, timeSteps, features = x.shape
# print(x.shape)
# print(y.shape)
# print(xval.shape)
# print(yval.shape)


In [120]:
stepOut = 2
features=1
stepsIn = 3
batchSize=200
valSize=100
a,b=3,6
func = lambda x: np.sqrt(x)

valGen = Generator(valSize,3000,func=func)
trainGen = Generator(batchSize, 2000,func=func)

msmodel = Sequential()
msmodel.add(LSTM(100,activation='relu',return_sequences=True,input_shape=(stepsIn,features)))
msmodel.add(LSTM(100,activation='relu'))
msmodel.add(Dense(stepOut))
msmodel.compile(loss='mse',optimizer='adam')

In [121]:
history = msmodel.fit(trainGen,validation_data=valGen,epochs=10,verbose=1,validation_steps=valSize,steps_per_epoch=batchSize)

Epoch 1/10
200/200 [==============================] - 6s 30ms/step - loss: 98.1180 - val_loss: 0.1211
Epoch 2/10
200/200 [==============================] - 6s 29ms/step - loss: 0.0091 - val_loss: 0.0077
Epoch 3/10
200/200 [==============================] - 6s 31ms/step - loss: 7.2368e-04 - val_loss: 0.0047
Epoch 4/10
  5/200 [..............................] - ETA: 4s - loss: 3.6776e-04

KeyboardInterrupt: 

In [125]:
samples = 10000
tx,ty = getTestMSData(uLimit=3000,nSamples=samples,func=func)
res = msmodel.predict(tx)
print(np.sum(abs(res-ty))/samples)
# print(tx)
# print(ty)
# print(res)

0.08457999997589837


In [140]:
stepsIn = 3
stepsOut = 2
features = 1
bs = 500
vs = 200
trainGen = Generator(bachSize=bs,uLimit=3000)
valGen = Generator(bachSize=vs,uLimit=4000)

lastModel = Sequential()
lastModel.add(LSTM(100, activation="relu", input_shape=(stepsIn, features)))
lastModel.add(RepeatVector(stepsOut))
lastModel.add(LSTM(100, activation='relu',return_sequences=True))
lastModel.add(TimeDistributed(Dense(1)))
lastModel.compile(loss='mse',optimizer='adam')

In [141]:
lastModel.fit(trainGen,validation_data=valGen,epochs=15,steps_per_epoch=bs,validation_steps=vs)

Epoch 1/15
500/500 [==============================] - 8s 16ms/step - loss: 19903234.0000 - val_loss: 57.9701
Epoch 2/15
500/500 [==============================] - 8s 15ms/step - loss: 26.5212 - val_loss: 45.5243
Epoch 3/15
500/500 [==============================] - 8s 15ms/step - loss: 21.1279 - val_loss: 56.8641
Epoch 4/15
500/500 [==============================] - 8s 16ms/step - loss: 20.9095 - val_loss: 66.4643
Epoch 5/15
500/500 [==============================] - 8s 15ms/step - loss: 20.6344 - val_loss: 58.2062
Epoch 6/15
500/500 [==============================] - 8s 15ms/step - loss: 20.7154 - val_loss: 43.1387
Epoch 7/15
500/500 [==============================] - 9s 17ms/step - loss: 20.6519 - val_loss: 51.2741
Epoch 8/15
500/500 [==============================] - 8s 16ms/step - loss: 20.3530 - val_loss: 48.5560
Epoch 9/15
500/500 [==============================] - 10s 20ms/step - loss: 20.6534 - val_loss: 68.9307
Epoch 10/15
500/500 [==============================] - 8s 16ms/ste

In [143]:
# tx,ty = valGen.__next__()
# res = lastModel.predict(tx[:1])
# print(tx[:1])
# print(ty[:1])
# print(res)
# lastModel.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_45 (LSTM)               (None, 100)               40800     
_________________________________________________________________
repeat_vector_4 (RepeatVecto (None, 2, 100)            0         
_________________________________________________________________
lstm_46 (LSTM)               (None, 2, 100)            80400     
_________________________________________________________________
time_distributed_3 (TimeDist (None, 2, 1)              101       
Total params: 121,301
Trainable params: 121,301
Non-trainable params: 0
_________________________________________________________________
